Une chaîne de Markov est un processus stochastique qui renseigne sur la probabilité d'observer une succession de $n$ événements. On considère ces événéments comme une séquence de $n$ variables aléatoires que l'on note $\boldsymbol{X} = (X_1, X_2, ..., X_n)$. Une chaîne de Markov peut être déterminée à partir de trois ensembles de paramètres :

- un ensemble de $p$ états $\mathcal{X} = \{s_1, s_2, s_3, ..., s_p\}$
- une matrice de transition $A$. Chaque élément $a_{ij}$ de A
représente la probabilité de transitionner de l'état i au temps t vers l'état j au temps t+1 :

$$
A = \begin{bmatrix}
a_{11} & a_{12} & \cdots & a_{1p}\\
a_{21} & a_{22} & \cdots & a_{2p}\\
\vdots & \vdots & \ddots & \vdots\\
a_{p1} & a_{p2} & \cdots & a_{pp}
\end{bmatrix}
$$

- un vecteur de distributions initiales pour chaque état $\Pi$:

$$
\Pi = \begin{bmatrix}
\pi_{1}\\
\pi_{2}\\
\vdots\\
\pi_{p}\\
\end{bmatrix}
$$

### Partie I - Compréhension du modèle

On se place désormais dans le cas d'application suivant. On souhaite développer un modèle de Markov de premier ordre capable de prédire la météo du prochain jour. Pour cela, on détermine trois états discrets : Nuageux, Pluvieux et Ensoleillé.

**Question 1** : Que devient l'ensemble des états $\mathcal{X}$ dans ce cas de figure ?

**Réponse** : $\mathcal{X} = \{s_s, s_c, s_r\}$.

**Question 2** : Que devient la matrice de transition A dans ce cas de figure ? Pour compléter la matrice, on utilisera des variables avec une notation sous la forme $a_{s_{from},s_{to}}$.

**Réponse** : $$
A = \begin{bmatrix}
a_{ss} & a_{sc} & a_{sr}\\
a_{cs} & a_{cc} & a_{cr}\\
a_{rs} & a_{rc} & a_{rr}
\end{bmatrix}
$$

**Question 3** : Que devient le vecteur de distributions initiales $\Pi$ dans ce cas de figure ? Pour compléter le vecteur, on utilisera des variables avec une notation sous la forme $\pi_{s_i}$.

**Réponse** :
$$
\Pi = \begin{bmatrix}
\pi_{s}\\
\pi_{c}\\
\pi_{r}\\
\end{bmatrix}
$$

Dans cette première partie, on considère que les paramètres du modèle sont donnés par des experts. On considère les éléments suivants :

- La probabilité de rester dans l'état Ensoleillé est de 0.5
- La probabilité de rester dans l'état Pluvieux est de 0.2
- La probabilité de rester dans l'état Nuageux est de 0.4

- La probabilité de transitionner de l'état Ensoleillé vers l'état Pluvieux est de 0.1
- La probabilité de transitionner de l'état Ensoleillé vers l'état Nuageux est de 0.4
- La probabilité de transitionner de l'état Pluvieux vers l'état Ensoleillé est de 0.2
- La probabilité de transitionner de l'état Pluvieux vers l'état Nuageux est de 0.6
- La probabilité de transitionner de l'état Nuageux vers l'état Ensoleillé est de 0.3
- La probabilité de transitionner de l'état Nuageux vers l'état Pluvieux est de 0.3

**Question 4** : Compléter le diagramme de la chaîne de Markov ci-dessous avec les valeurs numériques correspondantes :

**Réponse** :

In [1]:
from IPython.display import display, Image, HTML
html_code = '''
<div style="text-align: center;">
    <img src="MC_2.jpg" width="600">
</div>
'''
display(HTML(html_code))

**Question 5** : Remplacer les variables de la matrice de transition précédente par leur valeur numérique correspondante.

**Réponse** : $$
A = \begin{bmatrix}
0.5 & 0.4 & 0.1\\
0.3 & 0.4 & 0.3\\
0.2 & 0.6 & 0.2
\end{bmatrix}
$$

**Question 6** : Par construction, que vaut la somme des éléments de $\Pi$ ? Quelle contrainte similaire porte sur les éléments de la matrice A ? Vérifier que cette contrainte est satisfaite pour la matrice A.

**Réponse** : La somme des éléments de $\Pi$ est égale à 1.0 car il s'agit d'une somme de probabilités. La même contrainte porte sur les lignes de la matrice A : chaque ligne de la matrice A somme à 1.0.

Autrement dit, $\sum_{j=1}^p \Pi_j = 1$ et $\sum_{j=1}^p a_{i,j} = 1$

**Question 7** : Compléter maintenant la matrice $\Pi$ à l'aide des informations complémentaires suivantes :
- la probabilité que le premier jour soit Ensoleillé est de 0.7
- la probabilité que le premier jour soit Nuageux est de 0.2

**Réponse** : La matrice $\Pi$ étant stochastique, la somme de ses éléments doit être égale à 1. Par conséquent, la probabilité que le premier jour soit Pluvieux est de 0.1, et la matrice $\Pi$ peut être écrite sous la forme numérique suivante :

$$
\Pi = \begin{bmatrix}
0.7\\
0.2\\
0.1\\
\end{bmatrix}
$$

**Question 8** : Quelle est la propriété (ou hypothèse) de Markov dans le cadre de ce modèle d'ordre 1 ?

**Réponse** : La mémoire d'une chaîne de Markov d'ordre 1 n'est que d'un pas de temps. En d'autres termes :

$$ \mathcal{P}(X_t|X_{t-1}, X_{t-2}, ..., X_1) = \mathcal{P}(X_t|X_{t-1}) $$

### Partie II - Implémentation

Nous allons maintenant coder la classe MC1, qui permet de définir un objet correspondant à une chaîne de Markov d'ordre 1. 

- Première version : basée sur une seule séquence d'états

In [2]:
import numpy as np
from collections import Counter

In [3]:
class MC1_single_inputs():
    """
    Single inputs implementation of a first order Markov Chain.
    This algorithm takes a sequence of states as input, and can be used to infer the likelihood of an input sequence
    or estimate the probability of a state taking place in the next timestep, given an input sequence.

    """

    def encode_states(self, sequence):
        """Encodes a list of strings to numerical categorical values.

        Parameters
        ----------
        sequence : list of strings, of length n_states.
        A list of states.
        """
        self.counter_dict = Counter(sequence)
        self.unique_states = sorted(self.counter_dict)
        self.p = len(self.unique_states)
        self.states_encoding = {}
        self.states_decoding = {}
        for state_index, state in enumerate(self.unique_states):
            self.states_encoding[state] = state_index
            self.states_decoding[state_index] = state

    def compute_init_distribution_vector(self):
        """Computes the vector of initial distributions Pi.

        """
        self.init_dist = np.zeros(self.p)
        self.init_dist[self.states_encoding['sunny']] = 0.7
        self.init_dist[self.states_encoding['cloudy']] = 0.2
        self.init_dist[self.states_encoding['rainy']] = 0.1

    def compute_transition_matrix(self, sequence):
        """Computes the transition matrix A.

        Parameters
        ----------
        sequence : list of strings, of length n_states.
        A list of states.
        """
        self.transition_matrix = np.zeros((self.p, self.p))

        current_states = [self.states_encoding[state] for state in sequence[:-1]]
        next_states = [self.states_encoding[state] for state in sequence[1:]]
        np.add.at(self.transition_matrix, (current_states, next_states), 1)

    def normalize_transition_matrix(self):
        """Normalizes the previously computed transition matrix.

        """
        self.normalized_transition_matrix = self.transition_matrix.copy()
        self.normalized_transition_matrix /= self.normalized_transition_matrix.sum(axis=1)[:, np.newaxis]
    
    def fit(self, sequence):
        """Estimates all the parameters of the first order Markov chain.

        Parameters
        ----------
        sequence : list of strings, of length n_states.
        A list of states.
        """
        self.encode_states(sequence)
        self.compute_init_distribution_vector()
        self.compute_transition_matrix(sequence)
        self.normalize_transition_matrix()

    def predict_proba_next_state(self, sequence):
        """Predicts the probabilities of the next state occurring, given an input sequence of previous states.

        Parameters
        ----------
        sequence : list of strings, of length n_states.
        A list of states.

		Returns
		-------
		options_dict : dict of structure {state:probability}.
		A dictionary reporting the probability of each state taking place in the next timestep.
        """
        last_state = sequence[-1]
        state_encoding = self.states_encoding[last_state]
        state_tp = self.normalized_transition_matrix[state_encoding]
        options_dict = {}
        for index in list(self.states_decoding.keys()):
            options_dict[self.states_decoding[index]] = state_tp[index]
        
        return options_dict

    def compute_likelihood(self, sequence):
        """Computes the likelihood of an input sequence of states.

        Parameters
        ----------
        sequence : list of strings, of length n_states.
        A list of states.

		Returns
		-------
		likelihood : float.
		The likelihood of the input sequence.
        """
        encoded_states = [self.states_encoding[state] for state in sequence]
        previous_state = encoded_states[0]
        likelihood = self.init_dist[encoded_states[0]]
        for state in encoded_states[1:]:
            likelihood *= self.normalized_transition_matrix[previous_state, state]
            previous_state = state

        return likelihood

**Question 9** : On considère la séquence de 8 états suivante : $\boldsymbol{X} = [s_s, s_r, s_c, s_s, s_s, s_s, s_c, s_r]$. Quelle est la vraisemblance de cette succession d'états ?

**Réponse** : La vraisemblance vaut 0.002734375 (obtenue à l'aide de l'appel à la fonction compute_likelihood)

In [4]:
sequence_test = ['sunny', 'rainy', 'cloudy', 'sunny', 'sunny', 'sunny', 'cloudy', 'rainy']
MC1_si_model = MC1_single_inputs()
MC1_si_model.fit(sequence_test) 

In [5]:
MC1_si_model.states_encoding

{'cloudy': 0, 'rainy': 1, 'sunny': 2}

In [6]:
MC1_si_model.transition_matrix

array([[0., 1., 1.],
       [1., 0., 0.],
       [1., 1., 2.]])

In [7]:
MC1_si_model.normalized_transition_matrix

array([[0.  , 0.5 , 0.5 ],
       [1.  , 0.  , 0.  ],
       [0.25, 0.25, 0.5 ]])

In [8]:
MC1_si_model.compute_likelihood(sequence_test)

0.002734375

In [9]:
res =                     MC1_si_model.init_dist[2] 
res *= MC1_si_model.normalized_transition_matrix[2, 1]
res *=    MC1_si_model.normalized_transition_matrix[1, 0]
res *=       MC1_si_model.normalized_transition_matrix[0, 2]
res *=          MC1_si_model.normalized_transition_matrix[2, 2]
res *=             MC1_si_model.normalized_transition_matrix[2, 2]
res *=                MC1_si_model.normalized_transition_matrix[2, 0]
res *=                   MC1_si_model.normalized_transition_matrix[0, 1]
res

0.002734375

**Question 10** : On souhaite maintenant appliquer notre modèle à une tâche de prévision météo. Etant donné que l'on vient d'observer la séquence précédente, quelle est la probabilité que le prochain jour soit ensoleillé ? Nuageux ? Pluvieux ?

**Réponse** : La probabilité que le prochain jour soit nuageux est de 1. Les deux autres probabilités sont nulles (réponse obtenue à l'aide d'un appel à la fonction predict_proba_next_state).

In [10]:
MC1_si_model.predict_proba_next_state(sequence_test)

{'cloudy': 1.0, 'rainy': 0.0, 'sunny': 0.0}

**Question 11** : Même question, appliquée à la séquence suivante : $\boldsymbol{X_2} = [s_s, s_c, s_r]$. Que remarquez-vous ?

**Réponse** : Les probabilités sont identiques car nous utilisons une chaîne de Markov de premier ordre : seul le dernier état est informatif pour le modèle dans le cadre de cette question.

In [11]:
sequence_test_2 = ['sunny', 'cloudy', 'rainy']

In [12]:
MC1_si_model.predict_proba_next_state(sequence_test_2)

{'cloudy': 1.0, 'rainy': 0.0, 'sunny': 0.0}

**Question 12** : Même question, appliquée à la séquence suivante : $\boldsymbol{X_3} = [s_c, s_r, s_s, s_c, s_r, s_c]$. 

**Réponse** : L'idée ici est simplement d'insister sur le fait que seul le dernier état est informatif et qu'on n'a pas besoin de traiter toute la séquence pour obtenir la réponse souhaitée.

In [13]:
sequence_test_3 = ['cloudy', 'rainy', 'sunny', 'cloudy', 'rainy', 'cloudy']

In [14]:
MC1_si_model.predict_proba_next_state(sequence_test_3)

{'cloudy': 0.0, 'rainy': 0.5, 'sunny': 0.5}

- Deuxième version : basée sur une liste de séquence d'états (par exemple, une séquence correspond à l'évolution à New-York, une autre à Johannesburg, etc.)?

In [15]:
class MC1_multiple_inputs():
    """
    Multiple inputs implementation of a first order Markov Chain.
    This algorithm takes a sequence of states as input, and can be used to infer the likelihood of an input sequence
    or estimate the probability of a state taking place in the next timestep, given an input sequence.

    """
    
    def encode_states(self, sequences):
        """Encodes a list of strings to numerical categorical values.

        Parameters
        ----------
        sequences : list of list of strings, of length nb_sequences.
        A list of states.
        """
        joint_sequence = []
        for sequence in sequences:
            joint_sequence += sequence
            
        self.counter_dict = Counter(joint_sequence)
        self.unique_states = sorted(self.counter_dict)
        self.p = len(self.unique_states)
        self.states_encoding = {}
        self.states_decoding = {}
        for state_index, state in enumerate(self.unique_states):
            self.states_encoding[state] = state_index
            self.states_decoding[state_index] = state

    def compute_init_distribution_vector(self):
        """Computes the vector of initial distributions Pi.

        """
        self.init_dist = np.zeros(self.p)
        self.init_dist[self.states_encoding['sunny']] = 0.7
        self.init_dist[self.states_encoding['cloudy']] = 0.2
        self.init_dist[self.states_encoding['rainy']] = 0.1

    def compute_transition_matrix(self, sequences):
        """Computes the transition matrix A.

        Parameters
        ----------
        sequences : list of list of strings, of length nb_sequences.
        A list of states.
        """
        self.transition_matrix = np.zeros((self.p, self.p))

        for sequence in sequences:
            current_states = [self.states_encoding[state] for state in sequence[:-1]]
            next_states = [self.states_encoding[state] for state in sequence[1:]]
            np.add.at(self.transition_matrix, (current_states, next_states), 1)

    def normalize_transition_matrix(self):
        """Normalizes the previously computed transition matrix.

        """
        self.normalized_transition_matrix = self.transition_matrix.copy()
        self.normalized_transition_matrix /= self.normalized_transition_matrix.sum(axis=1)[:, np.newaxis]

    def fit(self, sequences):
        """Estimates all the parameters of the first order Markov chain.

        Parameters
        ----------
        sequences : list of list of strings, of length nb_sequences.
        A list of states.
        """
        self.encode_states(sequences)
        self.compute_init_distribution_vector()
        self.compute_transition_matrix(sequences)
        self.normalize_transition_matrix()

    def predict_proba_next_state(self, sequence):
        """Predicts the probabilities of the next state occurring, given an input sequence of previous states.

        Parameters
        ----------
        sequence : list of strings, of length n_states.
        A list of states.

		Returns
		-------
		options_dict : dict of structure {state:probability}.
		A dictionary reporting the probability of each state taking place in the next timestep.
        """
        last_state = sequence[-1]
        state_encoding = self.states_encoding[last_state]
        state_tp = self.normalized_transition_matrix[state_encoding]
        options_dict = {}
        for index in list(self.states_decoding.keys()):
            options_dict[self.states_decoding[index]] = state_tp[index]
        
        return options_dict

    def compute_likelihood(self, sequence):
        """Computes the likelihood of an input sequence of states.

        Parameters
        ----------
        sequence : list of strings, of length n_states.
        A list of states.

		Returns
		-------
		likelihood : float.
		The likelihood of the input sequence.
        """
        encoded_states = [self.states_encoding[state] for state in sequence]
        previous_state = encoded_states[0]
        likelihood = self.init_dist[encoded_states[0]]
        for state in encoded_states[1:]:
            likelihood *= self.normalized_transition_matrix[previous_state, state]
            previous_state = state

        return likelihood

In [16]:
sequences_test = [
                  ['sunny', 'cloudy', 'sunny', 'sunny', 'sunny', 'cloudy', 'cloudy', 'cloudy'], # Tunis
                  ['rainy', 'rainy', 'cloudy', 'sunny', 'cloudy', 'rainy'], # Bombay
                  ['sunny', 'sunny', 'cloudy', 'cloudy', 'rainy', 'cloudy', 'rainy', 'cloudy', 'sunny', 'rainy', 'cloudy', 'rainy', 'sunny'], # Bruxelles
                  ['rainy', 'rainy', 'rainy', 'cloudy', 'rainy', 'cloudy', 'cloudy', 'cloudy', 'cloudy', 'rainy'] # Paris
                 ]

MC1_mi_model = MC1_multiple_inputs()
MC1_mi_model.fit(sequences_test)

In [17]:
MC1_mi_model.states_encoding

{'cloudy': 0, 'rainy': 1, 'sunny': 2}

In [18]:
MC1_mi_model.normalized_transition_matrix

array([[0.4  , 0.4  , 0.2  ],
       [0.6  , 0.3  , 0.1  ],
       [0.5  , 0.125, 0.375]])

**Question 13** : On considère à nouveau notre séquence de 8 états précédente : $\boldsymbol{X} = [s_s, s_r, s_c, s_s, s_s, s_s, s_c, s_r]$. Quelle est maintenant la probabilité d'observer cette succession d'états ?

**Réponse** : 0.0002953125. La vraisemblance est plus faible que tout à l'heure (0.002734375), notamment car on a rajouté des villes où la succession de jours ensoleillés est plutôt rare.

In [19]:
MC1_mi_model.compute_likelihood(sequence_test)

0.0002953125

In [20]:
res =                     MC1_mi_model.init_dist[2] 
res *= MC1_mi_model.normalized_transition_matrix[2, 1]
res *=    MC1_mi_model.normalized_transition_matrix[1, 0]
res *=       MC1_mi_model.normalized_transition_matrix[0, 2]
res *=          MC1_mi_model.normalized_transition_matrix[2, 2]
res *=             MC1_mi_model.normalized_transition_matrix[2, 2]
res *=                MC1_mi_model.normalized_transition_matrix[2, 0]
res *=                   MC1_mi_model.normalized_transition_matrix[0, 1]
res

0.0002953125

**Question 14** Appliquer ce nouveau modèle aux séquences de test 1, 2 et 3 précédentes. Comment les probabilités d'observer chaque état ont-elles évolué ? 

**Réponse** : On a toujours une égalité pour les séquences de test 1 et 2, car on reste sur un modèle de Markov de premier ordre. Pour le reste, on a une distribution des probabilités plus nuancée que dans le cas précédent, car on a récolté plus d'échantillons : comme on raisonne de manière fréquentielle et qu'on a pu observer de plus nombreuses transitions distinctes, le modèle donne plus de chances à des situations malgré tout peu fréquentes de se produire.

In [21]:
MC1_mi_model.predict_proba_next_state(sequence_test)

{'cloudy': 0.6, 'rainy': 0.3, 'sunny': 0.1}

In [22]:
MC1_mi_model.predict_proba_next_state(sequence_test_2)

{'cloudy': 0.6, 'rainy': 0.3, 'sunny': 0.1}

In [23]:
MC1_mi_model.predict_proba_next_state(sequence_test_3)

{'cloudy': 0.4, 'rainy': 0.4, 'sunny': 0.2}